# Host Model
## 1 - Move your dataset to s3

In [ ]:
! aws s3 cp '../data/raw/amazon_reviews_multilingual_UK_v1_00.tsv' 's3://sagemaker-product-classification-sklearn/data/raw/amazon_reviews_multilingual_UK_v1_00.tsv'

## 2 - Create SKLearn 

In [ ]:
! pip3 install awscli==1.17 boto3==1.11 botocore==1.14 s3transfer==0.3 --user

In [15]:
import boto3
import re

import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn

In [95]:
role = get_execution_role()
bucket = 'sagemaker-product-classification-sklearn'
prefix = 'sagemaker/nb_classification'
script_path = 'nb-cls.py'
sess = sagemaker.Session()

In [97]:
# run the Scikit-Learn script
sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.m5.large",  # comment to local mode
    sagemaker_session=sess,  # comment to local mode
    # train_instance_type='local',  # uncomment to local mode
    role=role,
    source_dir='source'
)

## 3 - Fit the model

In [98]:
sklearn.fit({'train': 's3://sagemaker-product-classification-sklearn/data/raw/amazon_reviews_multilingual_UK_v1_00.tsv'})  # comment to local mode
#sklearn.fit({'train': 'file://data/raw/amazon_reviews_multilingual_UK_v1_00.tsv'})  # uncomment to local mode

2020-01-21 20:47:29 Starting - Starting the training job...
2020-01-21 20:47:30 Starting - Launching requested ML instances...
2020-01-21 20:48:27 Starting - Preparing the instances for training......
2020-01-21 20:49:16 Downloading - Downloading input data......
2020-01-21 20:50:15 Training - Training image download completed. Training in progress.2020-01-21 20:50:16,371 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-01-21 20:50:16,373 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-21 20:50:16,382 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-01-21 20:50:16,596 sagemaker-containers INFO     Module nb-cls does not provide a setup.py. 
Generating setup.py
2020-01-21 20:50:16,596 sagemaker-containers INFO     Generating setup.cfg
2020-01-21 20:50:16,596 sagemaker-containers INFO     Generating MANIFEST.in
2020-01-21 20:50:16,596 sagemaker-containers INFO     Installing mo

## 4 - Deploy

In [135]:
# run in local mode
#! docker container stop $(docker container ls -aq)
#! docker container rm $(docker container ls -aq)

In [100]:
# deploy the model to a SageMaker endpoint
predictor = sklearn.deploy(
    initial_instance_count=1,
    #instance_type="local"  # uncomment to local mode
    instance_type="ml.t2.medium"  # comment to local mode
)

-------------!

## 5 - Predict

In [134]:
import json
import numpy as np
import pandas as pd

sample_data = {
    'product_title': "Family Farm Seaside",
    'review_body': ",updated the game and it deleted the farm I'd spent over 2 days creating!<br /><br />Waste of my time!<br /><br />Was beginning to enjoy it as well but there goes that idea!",
    'review_headline': "stupid!"
}

model_input = [sample_data[x] for x in sample_data]
resp = predictor.predict(model_input)[0][0]
print(resp)

Mobile_Apps
